In [33]:
import folium
import zipfile
import io
from fastkml import kml
import pandas as pd
from io import StringIO
import csv
from tqdm import tqdm
import numpy as np

import pyvista as pv
pv.set_jupyter_backend('trame')

In [3]:
# read KML content 
kmz = zipfile.ZipFile("qfaults.kmz", 'r')  # extract zip file first, then read kmz file inside the extracted folder
kml_content = kmz.open('doc.kml', 'r').read()  # kml content
k = kml.KML()
k.from_string(kml_content)

In [4]:
docs = list(k.features())
folders=[]
for d in docs:
    folders.extend(list(d.features()))
records = []
for f in folders:
    records.extend(list(f.features()))
geoms = [element.geometry for element in records]

In [78]:
# print(dir(folders[0]))
print(folders[0].name)
print(folders[0].id)

Historical (150 years)
FeatureLayer0


In [20]:
names = [element.name for element in records]
print(len(set(names)))
print(len(name))

2107
112527


In [5]:
def get_header(record):
    tables = pd.read_html(StringIO(str(record.description)))
    data_table = list(tables[1][0])
    return data_table

def read_record(record):
    tables = pd.read_html(StringIO(str(record.description)))
    data_table = list(tables[1][1])
    return data_table

def get_all_data(record):
    data = []
    data.append(record.name)
    data.append(record.id)
    data.extend(read_record(record))
    data.append(record.geometry.wkt)
    return data

def get_all_headers(record):
    data = []
    data.append("Name")
    data.append("ID")
    data.extend(get_header(record))
    data.append("Geom")
    return data

print(get_all_headers(records[0]))
print(get_all_data(records[0]))
# headers = [len(get_header(element)) for element in records]

['Name', 'ID', 'Fault Name', 'Section Name', 'Fault ID', 'Section ID', 'Location', 'Linetype', 'Age', 'Dip Direction', 'Slip Rate(mm/year)', 'Slip Sense', 'Mapped Scale', 'Fault Class', 'Mapping Certainty', 'Average Strike', 'Total Fault Length (km)', 'Cooperator', 'Associated Earthquake', 'Date of Last Review', 'Fault url', 'symbology', 'Citation ID', 'Fault ID Number', 'Geom']
['San Andreas fault zone', 'ID_00000', 'San Andreas fault zone', 'Shelter Cove Section', '1', 'a', 'California', 'Inferred', 'historic', 'Vertical', 'Greater than 5.0 mm/yr', 'Right lateral', 'unspecified', 'A', 'Good', 'N12°W', '1082', 'California Geological Survey', 'San Francisco earthquake', '12/10/2002', 'https://earthquake.usgs.gov/cfusion/qfault/show_report_AB_archive.cfm?fault_id=1&section_id=a', 'historic Inferred', '1a', '1', 'MULTILINESTRING((-124.090694000324 40.11756400000441 0.0, -124.0905649995543 40.11569200027037 0.0, -124.0904559997744 40.11512000009119 0.0, -124.0901240004117 40.1146650003225

In [131]:
geoms = [element.geometry for element in records]  # extract geometry
# dir(geoms[0])

In [138]:
filename = 'Qfaults.csv'
with open(filename, 'w', newline="") as file:
    csvwriter = csv.writer(file)
    header = get_all_headers(records[0])
    csvwriter.writerow(header)
    for record in tqdm (records, desc="Writing File..."):
        data = get_all_data(record)
        csvwriter.writerow(data)
    # csvwriter.writerows(data) # 5. write the rest of the data

Writing File...: 100%|████████████████████████████████████████████████████████| 112527/112527 [05:02<00:00, 371.59it/s]


In [15]:
filter_name = "Calaveras fault zone"
filtered_records = list(filter(lambda record: (record.name == filter_name), records))  
filtered_records = list(map(get_all_data, filtered_records))
filtered_records = list(filter(lambda record: (record[8] == "historic"), filtered_records))  
# pv.Cone().plot()

In [72]:
def get_points(record):
    #last is lat,long
    multi_line_string = record[-1][17:-2]
    # MULTILINESTRING((-121.5036249996688 37.03746799973482 0.0, -121.503775000355 37.03769099972591 0.0))
    points = multi_line_string.split(',')
    lat_longs = []
    for point in points:
        lat_longs.append([float(x) for x in point.split(" ") if x]) #removes empty string
    lat_longs = np.array(lat_longs)
    return lat_longs
    # print(f"{lat_longs.shape}")


def get_cartesian(lat=None,lon=None,R = 6371): #in km:
    # lat, lon = np.deg2rad(lat), np.deg2rad(lon)
    # R = 6371 # radius of the earth
    x = R * np.cos(lat) * np.cos(lon)
    y = R * np.cos(lat) * np.sin(lon)
    z = R *np.sin(lat)
    return np.vstack((x,y,z)).T

all_points = get_points(filtered_records[4])
# print(all_points)
all_cartesian = get_cartesian(all_points[:,0],all_points[:,1])
print(all_cartesian.shape)

(5, 3)


In [104]:
def create_wall(lat_longs,diff=1,R = 6371,connectivity_delta=0):
    all_cartesian_top = get_cartesian(lat_longs[:,0],lat_longs[:,1],R+diff)
    all_cartesian_bottom = get_cartesian(lat_longs[:,0],lat_longs[:,1],R-diff)
    number_of_points,dim = all_cartesian_top.shape
    assert(dim==3)
    connectivity = []
    for i in range(number_of_points-1):
        current_top=i+connectivity_delta
        next_top=i+1+connectivity_delta
        current_bottom=number_of_points+i+connectivity_delta
        next_bottom=number_of_points+i+1+connectivity_delta
        connectivity.extend([4,current_top,next_top,next_bottom,current_bottom])
    return np.vstack((all_cartesian_top,all_cartesian_bottom)),connectivity

def center_points(vertices):
    center = np.average(vertices,axis=0)
    # print(vertices-center)
    return vertices-center

In [115]:
lat_longs = get_points(filtered_records[5])
vertices,connectivity = create_wall(lat_longs,diff=1)
vertices = center_points(vertices)
mesh = pv.PolyData(vertices, connectivity)
mesh.plot()

[[-1.67897981  0.16019123 -0.11886705]
 [ 0.66035465  0.45609916 -1.48817643]
 [-0.65998752 -0.45605272  1.48796154]
 [ 1.67861268 -0.16023767  0.11908194]]


Widget(value='<iframe src="http://localhost:61734/index.html?ui=P_0x24545c19220_6&reconnect=auto" class="pyvis…

In [114]:
# all_vertices = None
# all_connectivity = []
# current_num_points = 0
# for record in filtered_records:
#     lat_longs = get_points(record)
#     new_points,dim = lat_longs.shape
#     assert(dim==3)
#     vertices,connectivity = create_wall(lat_longs,diff=1,connectivity_delta=current_num_points)
#     if all_vertices is None:
#         all_vertices=vertices
#     else:
#         all_vertices=np.vstack((all_vertices,vertices))
#     all_connectivity.extend(connectivity)
#     current_num_points+=new_points

# all_vertices = center_points(all_vertices)
# mesh = pv.PolyData(all_vertices, connectivity)
# mesh.plot()


[[-2667.29374248 -1441.54966914  1663.79995071]
 [-2687.28582835 -1482.66068252  1683.64618674]
 [-2665.71497702 -1441.74145306  1665.01268256]
 ...
 [  598.09744415    72.50361453  -214.62898369]
 [  598.70563823    70.00851741  -213.51957337]
 [  598.65129084    71.83660988  -212.82666334]]


Widget(value='<iframe src="http://localhost:61734/index.html?ui=P_0x24545bb3620_5&reconnect=auto" class="pyvis…